# Embedding

In [1]:
!pip install --q ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s eta 0:00:00
time: 445 µs (started: 2024-07-02 02:48:52 +00:00)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

time: 13.4 s (started: 2024-07-02 02:48:52 +00:00)


## 1. 데이터 준비

In [3]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

time: 596 µs (started: 2024-07-02 02:49:43 +00:00)


In [4]:
# CountVectorizer
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# 숫자로 바로 바꿔버림 - token(단어)를 잘라서 각 단어를 컬럼 이름으로 씀
# 그 단어가 그 문장에 있으면 이를 1으로 함 -> 문자를 숫자화시킴, 많이 나온 순서대로 array를 줄을 세워서 알고리즘에 집어넣음
# -> 가장 간단한 방법 이외에도 Hash, Vector...등이 있음
# RNN에 넣을 것이므로 조금 다른 방법 사용

time: 415 µs (started: 2024-07-02 02:53:28 +00:00)


# 토큰나이저(Tokenizing)

In [6]:
# 과거형, 현재형, 동사 - 현재 진행형 등 -> 다 같은 취급?다른 취급?
# 영어는 단어 단위로 끊어냄 - 전처리 진행
from keras.preprocessing.text import Tokenizer
# 전체 단어 중 최상위 n개만 뽑아서 쓰겠다(num_words)
# oov_token : 100개 안에 못드는 놈들은 oov 처리를 함
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')
tokenizer.fit_on_texts(corpus) # 내부적으로 토큰화, 줄세우고, 필요없으면 oov 처리

time: 1.09 ms (started: 2024-07-02 03:00:56 +00:00)


In [7]:
tokenizer.word_index # 언어 사전

{'<OOV>': 1,
 'this': 2,
 'is': 3,
 'the': 4,
 'document': 5,
 'first': 6,
 'second': 7,
 'and': 8,
 'third': 9,
 'one': 10}

time: 4.35 ms (started: 2024-07-02 03:01:21 +00:00)


In [10]:
sequences = tokenizer.texts_to_sequences(corpus) # 숫자로 바꿔줌
sequences

[[2, 3, 4, 6, 5], [2, 5, 3, 4, 7, 5], [8, 2, 3, 4, 9, 10], [3, 2, 4, 6, 5]]

time: 9.3 ms (started: 2024-07-02 03:04:58 +00:00)


## 패딩

In [13]:
# 해당 숫자를 기존의 알고리즘에 넣을 수 있는데, 개수가 다름
# padding 해서 길이를 맞춰 줌
from keras.preprocessing.sequence import pad_sequences
# 전체 길이를 얼마로? : maxlen # 우리는 최대가 6개, 짧으면 긴게 잘림
# 앞에 0 :pre, 뒤 0 :post # truncating : 앞에?뒤에?를 자를지 결정
padded = pad_sequences(sequences,
              maxlen=6,
              padding = 'pre',
              truncating = 'pre')
padded

array([[ 0,  2,  3,  4,  6,  5],
       [ 2,  5,  3,  4,  7,  5],
       [ 8,  2,  3,  4,  9, 10],
       [ 0,  3,  2,  4,  6,  5]], dtype=int32)

time: 5 ms (started: 2024-07-02 03:09:33 +00:00)


In [ ]:
# RNN - 입력을 받아드리는 format으로 바꿔야 함
# RNN으로 보내려면 단어 하나를 몇 개의 숫자로 표현할지를 결정하는 임베딩 작업을 해야 함
# 쉬운 방법은 하나하나를 one-hot encoding 해주면 됨
# 지금은 짧으니 가능하지만, 데이터가 커지면?? 메모리 과부화됨, 학습도 잘 안됨
# 따라서 다른 방법을 택함

### one-hot encoding
* 각 단어를 벡터화 시킨다.

In [14]:
padded.shape

(4, 6)

time: 4.27 ms (started: 2024-07-02 03:13:15 +00:00)


In [16]:
from keras.utils import to_categorical

padded_o = to_categorical(padded)
padded_o

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0

time: 11.9 ms (started: 2024-07-02 03:14:36 +00:00)


In [ ]:
# LLM - 하나의 단어를 몇 개의 숫자로 표현, 어떻게 하면 비슷한 단어끼리 비숫한 숫자로 만들어 낼 것인지가 LLM의 메인
# word_to_back을 사용하기도 함
# filter를 학습이 잘되면 분류가 잘 될 것 - 어떤 숫자로 표현하면 좋을지 자체를 학습시킴